In [1]:
import win32com.client
import pyttsx3;
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Input

In [2]:
class Command():
    def __init__(self):
        outlook = win32com.client.Dispatch('outlook.application')
        self.mapi = outlook.GetNamespace("MAPI")
        
    def readLastInboxMessage(self):
        inbox = self.mapi.GetDefaultFolder(6)
        messages = inbox.Items
        message = messages.GetLast()
        return message.Body
    
    def readLastInboxSubject(self):
        inbox = self.mapi.GetDefaultFolder(6)
        messages = inbox.Items
        message = messages.GetLast()
        return message.Subject

    def getAttachmentofLastEmail(self):
        inbox = self.mapi.GetDefaultFolder(6)
        messages = inbox.Items
        message = messages.GetLast()
        return message.Attachments  
    
    def execute(self, commandType):
        if commandType == 1:
            return self.readLastInboxMessage()
        elif commandType == 2:
            return self.readLastInboxSubject()
        else:
            return None

In [3]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary = True)

In [4]:
model.most_similar("attachment")

[('attachments', 0.6731535196304321),
 ('Charge_Writ', 0.5680820941925049),
 ('attached', 0.5141735076904297),
 ('attach', 0.4919285774230957),
 ('attachement', 0.4828201234340668),
 ('base_weatherhead', 0.47360366582870483),
 ('postcard.exe', 0.4698409140110016),
 ('attaches', 0.45461004972457886),
 ('prosthetic_beak', 0.4472523629665375),
 ('Attachment', 0.44549378752708435)]

In [5]:
colnames = ["input","label"]
ds = pd.read_csv('synthetic_training_data.csv', usecols = colnames)
with open("stoplist.txt", "r") as F:
    stoplist = F.read()
    clean = []
for i in range(0, len(ds["input"])):
    clean.append([word for word in ds["input"][i].split() if word not in stoplist])

x_train = np.array([])
zero = np.zeros(shape = (1,300))

for i in range(0, len(ds["input"])):
    sentence = clean[i]
    vec = np.array([])
    for j in range(0,len(sentence)):
        word = model[sentence[j]]
        vec = np.append(vec, word)
    vec = vec.reshape(j+1,300)
    for k in range(j,19):
        vec = np.append(vec, zero)
    vec = np.array(vec)
    vec = vec.reshape(20,300)
    x_train = np.append(x_train, vec)
x_train = x_train.reshape(len(ds["input"]),20, 300)
print(x_train.shape)

(590, 20, 300)


In [6]:
num_classes = 3
temp = ds["label"]
y_train = []
for i in temp:
    current = [0] * num_classes
    current[i-1] = 1
    y_train.append(current)

y_train = np.array(y_train, dtype = int)
#print(y_train)

In [7]:
nnmodel = keras.Sequential()
nnmodel.add(keras.layers.InputLayer(input_shape = (20, 300)))
nnmodel.add(keras.layers.Flatten())
nnmodel.add(keras.layers.Dense(units = 300, activation = "relu"))
nnmodel.add(keras.layers.Dense(units = 250, activation = "relu"))
nnmodel.add(keras.layers.Dense(units = 100, activation = "relu"))
nnmodel.add(keras.layers.Dense(units = 20, activation = "relu"))
nnmodel.add(keras.layers.Dense(units = num_classes, activation = "softmax"))

optim = tf.keras.optimizers.Adam(learning_rate = 0.00001)

nnmodel.compile(optim, 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
nnmodel.fit(x_train, y_train, epochs=50, batch_size = 5, validation_split = 0.1)
nnmodel.summary()

Epoch 1/50
107/107 [==============================] - 1s 9ms/step - loss: 1.0745 - accuracy: 0.6064 - val_loss: 1.1002 - val_accuracy: 0.0339
Epoch 2/50
107/107 [==============================] - 1s 7ms/step - loss: 1.0158 - accuracy: 0.8719 - val_loss: 1.0955 - val_accuracy: 0.0678
Epoch 3/50
107/107 [==============================] - 1s 8ms/step - loss: 0.9368 - accuracy: 0.8851 - val_loss: 1.0875 - val_accuracy: 0.0678
Epoch 4/50
107/107 [==============================] - 1s 8ms/step - loss: 0.8425 - accuracy: 0.9002 - val_loss: 1.0878 - val_accuracy: 0.0847
Epoch 5/50
107/107 [==============================] - 1s 8ms/step - loss: 0.7433 - accuracy: 0.9134 - val_loss: 1.0746 - val_accuracy: 0.1017
Epoch 6/50
107/107 [==============================] - 1s 7ms/step - loss: 0.6485 - accuracy: 0.9247 - val_loss: 1.0628 - val_accuracy: 0.1017
Epoch 7/50
107/107 [==============================] - 1s 7ms/step - loss: 0.5601 - accuracy: 0.9322 - val_loss: 1.0716 - val_accuracy: 0.1186
Epoch 

In [10]:
#sentence = input("Ask the AI to do somehting")
sentence = "what was my latest body"
clean = []
clean = [word for word in sentence.split() if word not in stoplist]
print(clean)
vec = np.array([])
for j in range(0,len(clean)):
    word = model[clean[j]]
    vec = np.append(vec, word)
vec = vec.reshape(j+1,300)
for k in range(j,19):
    vec = np.append(vec, zero)
vec = np.array(vec)
vec = vec.reshape(1, 20, 300)

ar = np.array(nnmodel.predict(vec))
print(ar)
classnum =  np.argmax(ar) + 1
print(classnum)

emailController = Command()
print("Email Output: ")
emailController.execute(classnum)

['latest', 'body']
[[0.5515888  0.01888747 0.42952377]]
1
Email Output: 


'Revisit your favourite restaurants or try a dish from somewhere new. \r\n <https://s3.amazonaws.com/uber-static/emails/2016/10/halloweenrider/sp_12x12.png> \t  \t\r\n <http://email.uber.com/ss/c/lKT8ccCkHTtPXRlUdr2M3PTWD1XrrsuiXVNpxIMMrRkpnboC2Fsdovf95YuNHm-5/3ex/zqsIzTskQwmymCjAwTBFIQ/h0/3tnd03f3MCuY8OIAWwrTrLGhNl8f0rzG3Qh9a__Pv-c> \t\r\n <http://email.uber.com/ss/c/lKT8ccCkHTtPXRlUdr2M3PTWD1XrrsuiXVNpxIMMrRkpnboC2Fsdovf95YuNHm-5/3ex/zqsIzTskQwmymCjAwTBFIQ/h1/HLCXXC0QDzfTHfPl3WVojyWLnarxi_NDHIfqIHaNUhA> \t\r\n\r\nSatisfy your food cravings \r\n\r\nEnjoy more of the food you love, and so much more. \r\nOrder now  <https://ubereats.app.link/pdvNHMLAUab?%243p=a_custom_354088&~campaign=bloc_f367d4cd-9e80-4fbf-935e-049d0c51ffac&~channel=crm&~customer_keyword=2eecd197-6c4c-3b3c-8ed0-8963ad891d3e> \r\nGeneric \r\nRestaurants you rave about \t\r\n<http://email.uber.com/ss/c/lKT8ccCkHTtPXRlUdr2M3PTWD1XrrsuiXVNpxIMMrRkhFvAa2luIR5gQLm3LiNeaivePrto-FJSnZpeguusI92v_V88nAMEhMqKR1ywWnp1eBnVxGFyG1Jc